In [151]:
import pandas as pd
import numpy as np
import torch

In [152]:
sample = pd.read_csv("sample_submission.csv")
sample_df = pd.DataFrame(sample)
sample_df.head()

,row_id,document,token,label
0,0,7,9,B-NAME_STUDENT
1,1,7,10,I-NAME_STUDENT
2,2,7,482,B-NAME_STUDENT
3,3,7,483,I-NAME_STUDENT
4,4,7,741,B-NAME_STUDENT


In [153]:
train = pd.read_json("train.json")
train_df = pd.DataFrame(train)
train_df.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [154]:
test = pd.read_json("test.json")
test_df = pd.DataFrame(test)
test_df.head()

,document,full_text,tokens,trailing_whitespace
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal..."


In [155]:
class MostFrequentClass:
    
    def __init__(self, data: pd.DataFrame, target: str):
        self.target = target
        self.data = data
        self.pred_col1 = []
        self.pred_col2 = []
        self.values_dict = {}
        self.test_data = []
        self.TP, self.TN, self.FP, self.FN = 0, 0 , 0 , 0
        
    def train(self):
        col = self.data[self.target]
        
        for row_label_list in [x for x in col]:
            for value in row_label_list:
                self.values_dict[value] = self.values_dict.get(value, 0) + 1
        most_frequent = max(self.values_dict, key=lambda k: self.values_dict[k])
        
        self.data['most_frequent_class'] = pd.Series([most_frequent]*len(row_label_list) for row_label_list in [x for x in col])
            
        values_dict = self.values_dict
        if len(list(values_dict.keys())) > 1:
            values_dict.pop(most_frequent)
        second_frequent = max(values_dict, key=lambda k: values_dict[k])
        
        self.data['next_most_frequent_class'] = pd.Series([second_frequent]*len(row_label_list) for row_label_list in [x for x in col])
        
    def predict(self, test: pd.DataFrame):
        values_dict = {}
        col = test[self.target]
        for row_label_list in [x for x in col]:
            for value in row_label_list:
                values_dict[value] = values_dict.get(value, 0) + 1
            most_frequent = max(values_dict, key=lambda k: values_dict[k])
            self.pred_col1.append(most_frequent)
            if len(list(values_dict.keys())) > 1:
                values_dict.pop(most_frequent)
            second_frequent = max(values_dict, key=lambda k: values_dict[k])
            self.pred_col2.append(second_frequent)
            values_dict = {}
        for row_label_list in [x for x in col]:
            test['most_frequent_class'] = pd.Series([score]*len(row_label_list) for score in self.pred_col1)
            test['next_most_frequent_class'] = pd.Series([score]*len(row_label_list) for score in self.pred_col2)
        self.test_data = test
        return test

    def calculate_confusion_matrix(self):
        # Initialize counts
        TP = 0
        TN = 0
        FP = 0
        FN = 0

        for _, actual_row in self.data['labels'].items():  # Iterate over items of the Series
            index = 0  # Reset index for each row
            predicted_row = self.test_data['next_most_frequent_class'].loc[index]
    
            # Compare predicted values with actual values
            for actual_value, predicted_value in zip(actual_row, predicted_row):
                if actual_value == 1 and predicted_value == 1:
                    TP += 1
                elif actual_value == 0 and predicted_value == 0:
                    TN += 1
                elif actual_value == 0 and predicted_value == 1:
                    FP += 1
                elif actual_value == 1 and predicted_value == 0:
                    FN += 1
    
            index += 1
    
        self.TP=TP
        self.TN=TN
        self.FP=FP
        self.FN=FN


    def calculate_metrics(self):
        TP = self.TP
        TN = self.TN
        FP = self.FP
        FN = self.FN
        accuracy = (TP + TN) / (TP + TN + FP + FN)

        if TP + FP == 0:
            precision = 0
        else:
            precision = TP / (TP + FP)
    
        if TP + FN == 0:
            recall = 0
        else:
            recall = TP / (TP + FN)
    
        if precision + recall == 0:
            f1_score = 0
        else:
            f1_score = 2 * (precision * recall) / (precision + recall)
    
        return accuracy, precision, recall, f1_score
                
        
            
        

In [156]:
#split
def train_test_split(data: pd.DataFrame, train_percent: float):
    if train_percent < 0 or train_percent > 1:
        raise ValueError("train_percent must be between 0 and 1")

    test_percent = 1 - train_percent
    test_set = data[:int(len(data) * test_percent)]
    train_set = data[int(len(data) * test_percent):]
    return train_set, test_set

In [157]:
train_split, test_split = train_test_split(train_df, 0.9)

In [158]:
test_split

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...,...,...
675,8980,LEARNING LAUNCH\n\nCHALLENGE\n\nIt’s the begin...,"[LEARNING, LAUNCH, \n\n, CHALLENGE, \n\n, It, ...","[True, False, False, False, False, False, True...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
676,8982,Visualization tool – Sanne Aarts Challenge ...,"[Visualization, tool, –, Sanne, Aarts, , Ch...","[True, True, True, True, True, False, True, Tr...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
677,8984,1. Challenge : How do we make our customers ad...,"[1, ., Challenge, :, How, do, we, make, our, c...","[False, True, True, True, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
678,8986,MIIND MAPPING\n\nChallenge: As a student I ha...,"[MIIND, MAPPING, \n\n, Challenge, :, , As, a,...","[True, False, False, False, True, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [159]:
MFC = MostFrequentClass(train_split, 'labels')
MFC.train()
MFC.data

C:\Users\HP\AppData\Local\Temp\ipykernel_8260\157145585.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['most_frequent_class'] = pd.Series([most_frequent]*len(row_label_list) for row_label_list in [x for x in col])
C:\Users\HP\AppData\Local\Temp\ipykernel_8260\157145585.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['next_most_frequent_class'] = pd.Series([second_frequent]*len(row_label_list) for row_label_list in [x for x in col])


,document,full_text,tokens,trailing_whitespace,labels,most_frequent_class,next_most_frequent_class
680,8988,We believe that all kids are naturally creativ...,"[We, believe, that, all, kids, are, naturally,...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
681,8989,Reflection - Mind Mapping\n\nChallenge\n\nI am...,"[Reflection, -, Mind, Mapping, \n\n, Challenge...","[True, True, True, False, False, False, False,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
682,8993,Visualization\n\nChallenge\n\nSince I am an ev...,"[Visualization, \n\n, Challenge, \n\n, Since, ...","[False, False, False, False, True, True, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
683,8995,Visualization\n\nChallenge\n\nWhen I worked as...,"[Visualization, \n\n, Challenge, \n\n, When, I...","[False, False, False, False, True, True, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
684,8996,Visualizations are one of the important parts ...,"[Visualizations, are, one, of, the, important,...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
...,...,...,...,...,...,...,...
6802,22678,EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE My w...,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",NaN,NaN
6803,22679,Why Mind Mapping?\n\nMind maps are graphical r...,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",NaN,NaN
6804,22681,"Challenge\n\nSo, a few months back, I had chos...","[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",NaN,NaN
6805,22684,Brainstorming\n\nChallenge & Selection\n\nBrai...,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",NaN,NaN


In [160]:
pred = MFC.predict(test_split)
pred

C:\Users\HP\AppData\Local\Temp\ipykernel_8260\157145585.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['most_frequent_class'] = pd.Series([score]*len(row_label_list) for score in self.pred_col1)
C:\Users\HP\AppData\Local\Temp\ipykernel_8260\157145585.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['next_most_frequent_class'] = pd.Series([score]*len(row_label_list) for score in self.pred_col2)
C:\Users\HP\AppData\Local\Temp\ipykernel_8260\157145585.py:43: SettingWithCopyWarning: 
A value

,document,full_text,tokens,trailing_whitespace,labels,most_frequent_class,next_most_frequent_class
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
...,...,...,...,...,...,...,...
675,8980,LEARNING LAUNCH\n\nCHALLENGE\n\nIt’s the begin...,"[LEARNING, LAUNCH, \n\n, CHALLENGE, \n\n, It, ...","[True, False, False, False, False, False, True...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
676,8982,Visualization tool – Sanne Aarts Challenge ...,"[Visualization, tool, –, Sanne, Aarts, , Ch...","[True, True, True, True, True, False, True, Tr...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
677,8984,1. Challenge : How do we make our customers ad...,"[1, ., Challenge, :, How, do, we, make, our, c...","[False, True, True, True, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
678,8986,MIIND MAPPING\n\nChallenge: As a student I ha...,"[MIIND, MAPPING, \n\n, Challenge, :, , As, a,...","[True, False, False, False, True, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


# Evaluation

# Submission

In [181]:
from itertools import chain
for i in pred['tokens']:
    document = [[doc]*len(i) for doc in pred['document']]
document = list(chain.from_iterable(document))    
tokens = [temp2 for temp in pred['tokens'] for temp2 in temp]
labels = [temp2 for temp in pred['labels'] for temp2 in temp]
most_f = [temp2 for temp in pred['most_frequent_class'] for temp2 in temp]
next_most_f = [temp2 for temp in pred['next_most_frequent_class'] for temp2 in temp]


In [182]:
df = pd.DataFrame()
df['document'] = pd.Series(document)
df['tokens'] = pd.Series(tokens)
df['labels'] = pd.Series(labels)
df['most_frequent_class'] = pd.Series(most_f)

In [183]:
df

,document,tokens,labels,most_frequent_class
0,7,Design,O,O
1,7,Thinking,O,O
2,7,for,O,O
3,7,innovation,O,O
4,7,reflexion,O,O
...,...,...,...,...
550795,8987,NaN,NaN,O
550796,8987,NaN,NaN,O
550797,8987,NaN,NaN,O
550798,8987,NaN,NaN,O


In [184]:
# df.to_csv("submissions-mfc.csv", index=True)

In [185]:
df['next_most_frequent_class'] = pd.Series(next_most_f)

In [186]:
# df.to_csv("submissions-nmfc.csv", index=True)

# Ground truth to_csv

In [187]:
# df = pd.DataFrame()
# df['document'] = pd.Series(document)
# df['tokens'] = pd.Series(tokens)
# df['labels'] = pd.Series(labels)
# df.to_csv("evaluations.csv", index=True)

# Conf Matix

In [188]:
# accuracy, precision, recall, f1score = MFC.calculate_metrics()

In [189]:
# from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# mlb = MultiLabelBinarizer()
acc1 = accuracy_score(df['labels'], df['most_frequent_class'])
acc1

TypeError: '<' not supported between instances of 'float' and 'str'